In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Shareddrives/USV_Project_2021/E:/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1ZAaPx3ZyueNJEhIpm0_6D9KpkprurC0K/E:


In [2]:
import numpy as np
import PIL.Image as Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
from tensorflow.keras.preprocessing import image
import pandas as pd
import os
import xlrd
import math
import scipy
import scipy.io.wavfile as wavfile
from scipy.signal import butter, lfilter, freqz
import librosa
import librosa.display
import cv2

In [3]:
def butter_highpass_filter(cutoff_H, fs, order):
  nyq = 0.5*fs
  normal_cutoff = cutoff_H/nyq
  b, a = butter(order, normal_cutoff, btype='high', analog=False)
  return a, b

class sample:
  def __init__(self, mother, name, sex, age, matgen, pupgen, rec_num, syls, time_between):
    self.mother = mother
    self.name = name
    self.sex = sex
    self.age = age
    self.matgen = matgen
    self.pupgen = pupgen
    self.rec_num = rec_num
    self.syls = syls
    self.time_between = time_between

In [4]:
model_path = '/content/drive/Shareddrives/USV_Project_2021/E:/model_weights.h6'
model = keras.models.load_model(model_path, custom_objects={'KerasLayer':hub.KerasLayer})

fs = 250000
order = 6
cutoff_H = 30*10**3
c, d = butter_highpass_filter(cutoff_H, fs, order)

%cd /content/drive/MyDrive/final_project/
data_table = xlrd.open_workbook('New Segmentatio Data For Final Class_1.xlsx').sheet_by_index(0)
age = data_table.col_values(6, 1)
matgen = data_table.col_values(2, 1)
pupgen = data_table.col_values(5, 1)
mother = data_table.col_values(1, 1)
name = data_table.col_values(3, 1)
sex = data_table.col_values(4, 1)
session = data_table.col_values(7, 1)
#syllable = data_table.col_values(14, 1)
rec_num = data_table.col_values(8, 1)
start = data_table.col_values(9, 1)
finish = data_table.col_values(10, 1)

/content/drive/MyDrive/final_project


In [5]:
samples = []
sr = 250000
max_time = 0.25
pred = []
timeB = []
time_diff = []
start_arr = np.array(start) #converting to array
finish_arr = np.array(finish)
time_diff = np.subtract(finish_arr, start_arr) #calculating each syllable length
for i in range(len(mother)):
  path = '/content/drive/MyDrive/USV Recordings/2022/{}_{}/{}_{}/day_{}/session{}/{}.wav'.format(mother[i], matgen[i], name[i], pupgen[i], int(age[i]), int(session[i]), rec_num[i]) #find path of each recording
  if not os.path.exists('{}'.format(path)):
    path = '/content/drive/MyDrive/USV Recordings/2022/{}_{}/{}_{}/day_{}/session{}/{}.WAV'.format(mother[i], matgen[i], name[i], pupgen[i], int(age[i]), int(session[i]), rec_num[i])
    if not os.path.exists('{}'.format(path)):
      print(i)
      continue
  if i>0 and (rec_num[i] != rec_num[i-1] or name[i] != name[i-1]):
    recording = sample(mother[i-1], name[i-1], sex[i-1], age[i-1], matgen[i-1], pupgen[i-1], rec_num[i-1], pred, timeB)
    samples.append(recording)
    pred = []
    timeB = []
  if time_diff[i] < max_time:
    rec, rate = librosa.load(path, sr) #opens recordings and sample rate, add sample rate of USVs
    temp = (max_time - time_diff[i])/2
    silence = np.zeros(round((temp)*rate))
    trimmed = rec[round((start[i])*rate):round((finish[i])*rate)] #trimming the syllables according to start and fin poitns from excel
    syl = np.append(silence, trimmed) #normalizing length to max syllable
    syl = np.append(syl, silence)
  else:
    rec, rate = librosa.load(path, sr)
    syl = rec[round((start[i])*rate):round((finish[i])*rate)]
  #pre processing
  syl = lfilter(d, c, syl) #hpf
  #STFT creation
  D = np.abs(librosa.stft(syl, n_fft=2048, hop_length=128, win_length=512, window='hamming')) #creating stft and using absolute value for spectrogram
  D = cv2.resize(D, dsize=(128, 128), interpolation=cv2.INTER_CUBIC) #resizing the spectograms for usage in the model. INTER_CUBIC – a bicubic interpolation over 4×4 pixel neighborhood
  D = D - 0.02*np.mean(D)
  D = np.repeat(D[:, :, np.newaxis], 3, -1)
  D = D.astype('float32')
  D = D/255
  D = image.img_to_array(D)
  D = np.expand_dims(D, axis=0)
  predict = model.predict(D)
  pred.append(predict)
  if len(pred) > 1:
    timeB.append(start[i] - finish[i-1])

In [ ]:
print(samples[100].syls)

[array([[0.10424803, 0.02017718, 0.10995657, 0.04979197, 0.00645146,
        0.04784222, 0.00493989, 0.01105884, 0.63128066, 0.01425312]],
      dtype=float32), array([[1.4041255e-03, 9.1969721e-02, 1.5656819e-03, 9.0126616e-01,
        5.3496141e-04, 4.7518732e-04, 4.2291899e-04, 2.2549591e-04,
        1.6574243e-03, 4.7824337e-04]], dtype=float32), array([[0.01140546, 0.68735087, 0.0199605 , 0.24373047, 0.00204667,
        0.00183484, 0.00313926, 0.00193183, 0.02591688, 0.00268325]],
      dtype=float32)]


In [ ]:
samples = np.array(samples)
np.save('model_prediction_recordings_New_3B.npy', samples)

In [ ]:
'''
i = 731
pred = []
timeB = 1
sr = 250000 
time_diff = []
start_arr = np.array(start) #converting to array
finish_arr = np.array(finish)
time_diff = np.subtract(finish_arr, start_arr)
max_time = 0.25
path = '/content/drive/Shareddrives/TEST_RECORDINGS/Final_Project/Recordings_ella_ayelet/total_data/{}/{}/{}.wav'.format(mother[i], name[i], rec_num[i])
if time_diff[i] < max_time:
  rec, rate = librosa.load(path, sr) #opens recordings and sample rate, add sample rate of USVs
  temp = (max_time - time_diff[i])/2
  silence = np.zeros(round((temp)*rate))
  trimmed = rec[round((start[i])*rate):round((finish[i])*rate)] #trimming the syllables according to start and fin poitns from excel
  syl = np.append(silence, trimmed) #normalizing length to max syllable
  syl = np.append(syl, silence)
else:
  rec, rate = librosa.load(path, sr)
  syl = rec[round((start[i])*rate):round((finish[i])*rate)]
#pre processing
syl = lfilter(d, c, syl) #hpf

D = np.abs(librosa.stft(syl, n_fft=2048, hop_length=128, win_length=512, window='hamming')) #creating stft and using absolute value for spectrogram
D = cv2.resize(D, dsize=(128, 128), interpolation=cv2.INTER_CUBIC) #resizing the spectograms for usage in the model. INTER_CUBIC – a bicubic interpolation over 4×4 pixel neighborhood
D = D - 0.02*np.mean(D)
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.pcolormesh(20*np.log10(D))
D = np.repeat(D[:, :, np.newaxis], 3, -1)
D = D.astype('float32')
D = D/255
D = image.img_to_array(D)
D = np.expand_dims(D, axis=0)
predict = model.predict(D)
pred.append(predict)
pred.append(predict)
sample1 = sample(mother[i], name[i], sex[i], age[i], matgen[i], pupgen[i], rec_num[i], pred, timeB)
#pred = np.where(pred == np.max(pred))
print(sample1.syls)
'''

"\ni = 731\npred = []\ntimeB = 1\nsr = 250000 \ntime_diff = []\nstart_arr = np.array(start) #converting to array\nfinish_arr = np.array(finish)\ntime_diff = np.subtract(finish_arr, start_arr)\nmax_time = 0.25\npath = '/content/drive/Shareddrives/TEST_RECORDINGS/Final_Project/Recordings_ella_ayelet/total_data/{}/{}/{}.wav'.format(mother[i], name[i], rec_num[i])\nif time_diff[i] < max_time:\n  rec, rate = librosa.load(path, sr) #opens recordings and sample rate, add sample rate of USVs\n  temp = (max_time - time_diff[i])/2\n  silence = np.zeros(round((temp)*rate))\n  trimmed = rec[round((start[i])*rate):round((finish[i])*rate)] #trimming the syllables according to start and fin poitns from excel\n  syl = np.append(silence, trimmed) #normalizing length to max syllable\n  syl = np.append(syl, silence)\nelse:\n  rec, rate = librosa.load(path, sr)\n  syl = rec[round((start[i])*rate):round((finish[i])*rate)]\n#pre processing\nsyl = lfilter(d, c, syl) #hpf\n\nD = np.abs(librosa.stft(syl, n_fft